In [1]:
import functools
#functools is a module for higher order functions and operations on callable objects

In [8]:
from functools import reduce
# reduce is a function that applies a function of two arguments cumulatively to the items of an iterable, 
# in succession from left to right, so as to reduce the iterable to a single value

#example of reduce
reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])#calculates ((((1+2)+3)+4)+5)

func = lambda x,y: x if x > y else y
reduce(func, [47,11,42,102,13])

addList = lambda x,y: x+y
reduce(addList,[[1,2],[3,4],[5,6]])


[1, 2, 3, 4, 5, 6]

In [3]:
from functools import partial
#partial is a function that takes a function and some arguments and returns a new function that behaves like the original function
#but with some of the arguments already set

def multiply(x, y):
    return x * y

# create a new function that multiplies by 2
dbl = partial(multiply,2)#2 replaces x
print(dbl(4))

from functools import partialmethod
#partialmethod is a function that behaves like partial, but it works with methods

class Cell:
    def __init__(self):
        self._alive = False
    def set_state(self, state):
        self._alive = bool(state)
    def get_state(self):
        return self._alive
    set_alive = partialmethod(set_state, True)
    set_dead = partialmethod(set_state, False)
    
c = Cell()
print(c.get_state())
c.set_alive()
print(c.get_state())

8


In [11]:
from functools import lru_cache
#lru_cache is a decorator that caches the results of the function it decorates
#it can save time when an expensive function is periodically called with the same arguments

@lru_cache(maxsize=32)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)



#compare with a version that does not use lru_cache
def fib_no_cache(n):
    if n < 2:
        return n
    return fib_no_cache(n-1) + fib_no_cache(n-2)

import time
start = time.time()
print(fib(30))
print("Time taken with cache:", time.time()-start)

start = time.time()
print(fib_no_cache(30))#because of the lack of caching, this is much slower than the previous version
print("Time taken without cache:", time.time()-start)


832040
Time taken with cache: 0.00047206878662109375
832040
Time taken without cache: 0.24500298500061035


In [ ]:
from functools import wraps

def my_decorator(f):
    @wraps(f)#wraps is a decorator that copies the metadata of the original function to the decorated function
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')
    
example()
print(example.__name__)

#another example comparing with and without wraps
def my_decorator(f):
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()
print(example.__name__)

#why __name__ is different?
#because the metadata of the original function is not copied to the decorated function without wraps
#the metadata is the information of the original function(function under @), 
# such as __name__, __doc__, __module__, __annotations__, __dict__, __qualname__
# when do we think the metadata is important?
# example: when we use the function in the interactive mode, we can use the metadata to get the information of the function
#example:



In [17]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Calling decorated function")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def say_hello():
    """This function says hello"""
    print("Hello!")

def my_decorator2(func):
    def wrapper(*args, **kwargs):
        print("Calling decorated function")
        return func(*args, **kwargs)
    return wrapper

@my_decorator2
def say_goodbye():
    """This function says goodbye"""
    print("Goodbye!")


# Without functools.wraps
# say_hello.__name__ would be 'wrapper' instead of 'say_hello'
# say_hello.__doc__ would be None instead of 'This function says hello'

print(say_hello.__name__)  # Outputs: say_hello
print(say_hello.__doc__)   # Outputs: This function says hello
say_hello()
print()
print(say_goodbye.__name__)  # Outputs: wrapper
print(say_goodbye.__doc__)   # Outputs: None
say_goodbye()

say_hello
This function says hello
Calling decorated function
Hello!

wrapper
None
Calling decorated function
Goodbye!


In [5]:
from functools import singledispatch

#it seems like overloading in C++

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

@fun.register #register is a method of singledispatch
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)
    
@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun("Hello, world.")
fun("test.", verbose=True)
fun(42)
fun(42, verbose=True)
fun([1, 2, 3])
fun([1, 2, 3], verbose=True)



Hello, world.
Let me just say, test.
42
Strength in numbers, eh? 42
0 1
1 2
2 3
Enumerate this:
0 1
1 2
2 3


In [7]:
from functools import total_ordering

#total_ordering is a class decorator that fills in missing ordering methods
#it takes a class that provides some of the methods and adds the rest of them

@total_ordering
class Student:
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) <
                (other.lastname.lower(), other.firstname.lower()))
        
#example of using total_ordering
#the class Student only provides __eq__ and __lt__ methods
#total_ordering will provide the rest of the methods
# __ne__ : not equal
# __gt__ : greater than
# __le__ : less than or equal
# __ge__ : greater than or equal

sdu = Student()
sdu.lastname = "Doe"
sdu.firstname = "John"

sde = Student()
sde.lastname = "Doe"
sde.firstname = "Jane"

print(sdu == sde)
print(sdu < sde)
print(sdu > sde)
print(sdu <= sde)


False
False
True
False
